In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
from pathlib import Path
import sys

import cv2
import dill as pickle
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from PIL import Image
import plotly.graph_objects as go
import pyransac3d as pyrsc
import pyrender
from tqdm import tqdm
import trimesh
import torch
import visu3d as v3d
os.environ["PYOPENGL_PLATFORM"] = "egl"

sys.path.append(os.path.join(os.getcwd(), "dust3r"))
from dust3r.inference import inference
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode
from lang_sam import LangSAM

from barrelnet.pointnet.pointnet_utils import PointNetEncoder, feature_transform_reguliarzer
from barrelnet.pointnet.barrelnet import BarrelNet
from barrelnet.pointnet.data import pts2inference_format
from barrelnet.dust3r_utils import save_dust3r_outs, read_dust3r, resize_to_dust3r
from barrelnet.langsam_utils import display_image_with_masks, display_image_with_boxes
from barrelnet.utils import cmapvals, segment_pc_from_mask, get_local_plane_mask, rotate_pts_to_ax
from barrelnet.synthbarrel import random_cylinder_vol, random_cylinder_surf, monte_carlo_volume_ratio, generate_oriented_barrel, get_cyl_endpoints, get_cylinder_surf

In [ ]:
# image_dir = Path("data/dive8-barrel-10-45-less")
# image_dir = Path("data/barrel1-1sec-middle")
# image_dir = Path("data/barrelddt1")
image_dir = Path("data/barrel2-5sec-contrast")

if not image_dir.exists():
    raise FileNotFoundError(f"Image directory {image_dir} not found.")

# H, W = (875, 1920)
H, W = (224, 512)
reconstr_dir = Path(f"results/{image_dir.name}-reconstr")
mask_dir = reconstr_dir / "masks"
mask_dir.mkdir(parents=True, exist_ok=True)
maskcomp_dir = reconstr_dir / "image_with_masks"
maskcomp_dir.mkdir(parents=True, exist_ok=True)
ply_dir = reconstr_dir / "pc_plys"
ply_dir.mkdir(parents=True, exist_ok=True)
resizeimg_dir = reconstr_dir / "resized"
resizeimg_dir.mkdir(parents=True, exist_ok=True)
text_prompt = "underwater barrel"
imgpaths = sorted(image_dir.glob("*.jpg"))
dust3rout_path = reconstr_dir / "dust3r_out.pth"

with open(reconstr_dir / "rotatedpts.pickle", "rb") as f:
    rotpcdict = pickle.load(f)

pc = v3d.Point3d(p=rotpcdict["p"], rgb=rotpcdict["rgb"])
T = rotpcdict["T"]
_, _, v3dcams = read_dust3r(dust3rout_path)
v3dcams = v3dcams.apply_transform(v3d.Transform.from_matrix(T))
pc_idx = 0
v3dcam = v3dcams[pc_idx]
v3d.make_fig([pc, v3dcam])

In [ ]:
plt.imshow(v3dcam.render(pc))

In [ ]:
camera_pose = np.eye(4)
camera_pose[:3, 3] = v3dcam.world_from_cam.t
camera_pose[:3, :3] = v3dcam.world_from_cam.R
K = v3dcam.spec.K
pyrcam = pyrender.camera.IntrinsicsCamera(K[0, 0], K[1, 1], K[0, 2], K[1, 2])

In [ ]:
Tx180 = np.array([
    [1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, 1]
])
camera_pose = np.eye(4)
camera_pose[:3, 3] = v3dcam.world_from_cam.t
camera_pose[:3, :3] = v3dcam.world_from_cam.R
camera_pose = camera_pose @ Tx180
K = v3dcam.spec.K
pyrcam = pyrender.camera.IntrinsicsCamera(K[0, 0], K[1, 1], K[0, 2], K[1, 2])

pcmesh = pyrender.Mesh.from_points(pc.p, colors=pc.rgb)
# m = pyrender.Mesh.from_points(cylpts, colors=cylcols)
cylmesh = pyrender.Mesh.from_trimesh(trimesh.creation.cylinder(radius=0.01, height=0.02, sections=10))
cylpose = np.eye(4)
cylpose[:3, 3] = np.array([-0.01, 0.2, 0.05])
scene = pyrender.Scene()
scene.add(pcmesh)
scene.add(cylmesh, pose=cylpose)
scene.add(pyrcam, pose=camera_pose)
light = pyrender.SpotLight(color=np.ones(3), intensity=1.0,
                            innerConeAngle=np.pi/16.0,
                            outerConeAngle=np.pi/6.0)
scene.add(light, pose=camera_pose)
r = pyrender.OffscreenRenderer(W, H)
color, depth = r.render(scene)
r.delete()
plt.imshow(color)